## Load Dataset

In [1]:
import os
import diffusers
import torch

import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt

from urllib.request import urlopen 
import json 

os.environ['HF_HOME'] = "/run/media/anton/hdd/hf"

In [2]:
from diffusers import DiffusionPipeline
import PIL.Image
import numpy as np

from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="/run/media/anton/hdd/data/audio-diffuser", )

/home/anton/source/torch-env/lib/python3.12/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


Resolving data files:   0%|          | 0/888 [00:00<?, ?it/s]

In [3]:
from torchvision import transforms

batch_size=8

preprocess = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Grayscale(),
            transforms.Normalize([0.5], [0.5]),
        ]
    )

def transform(examples):
    images = [preprocess(image.convert("RGB")) for image in examples["image"]]
    return {"images": images}

dataset.set_transform(transform)
train_dataloader = torch.utils.data.DataLoader(dataset["train"], batch_size=batch_size, shuffle=True)

## Define VAE Model

In [4]:
from diffusers.models import AutoencoderKL

model = AutoencoderKL(
    in_channels=1,  
    out_channels=1,  
    latent_channels=3,
    layers_per_block=2,  
    block_out_channels=(64, 64, 64, 64,),  # the number of output channels for each UNet block
    down_block_types=(
        "DownEncoderBlock2D",
        "DownEncoderBlock2D",
        "DownEncoderBlock2D",  
        "DownEncoderBlock2D",
    ),
    up_block_types=(
        "UpDecoderBlock2D",  
        "UpDecoderBlock2D",  
        "UpDecoderBlock2D",
        "UpDecoderBlock2D",
    ),
)

In [5]:
from diffusers.optimization import get_constant_schedule
from accelerate import Accelerator

lr=1e-3
epochs=150

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

lr_scheduler = get_constant_schedule(optimizer=optimizer)

accelerator = Accelerator(mixed_precision="fp16", gradient_accumulation_steps=1)

model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, lr_scheduler
)

In [6]:
batch = next(iter(train_dataloader))

In [7]:
x = batch['images']

q = model.encode(x).latent_dist

loss_kl = -0.5 * torch.sum(1 + q.logvar - q.mean.pow(2) - q.logvar.exp())

x_pred = model.decode(q.sample()).sample

loss_recon = torch.nn.functional.mse_loss(x_pred, x)

loss = loss_recon + loss_kl

In [8]:
with torch.no_grad():
    torch.cuda.empty_cache()

In [9]:
import torch.nn.functional as F

for epoch in range(epochs):
    for batch_idx, batch in enumerate(train_dataloader):

        x = batch['images']
        
        with accelerator.accumulate(model):
            q = model.encode(x).latent_dist

            loss_kl = -0.5 * torch.sum(1 + q.logvar - q.mean.pow(2) - q.logvar.exp())
            
            x_pred = model.decode(q.sample()).mode
            
            loss_recon = F.mse_loss(x_pred, x)
            
            loss = loss_recon + loss_kl

            
            accelerator.backward(loss)

            # accelerator.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        with torch.no_grad():
            torch.cuda.empty_cache()
        
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{batch_idx+1}/{len(train_dataloader)}], Loss: {loss.item():.4f}')

OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 